In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/TresAnonyme_331', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)
print(anon.head(50))

     id                date  latitude  longitude  week
0    67 2015-04-12 17:17:00     4.836     45.767    15
1    59 2015-04-21 17:10:00     2.313     49.874    17
5    75 2015-04-03 11:51:00     4.863     45.780    14
7    65 2015-04-04 01:27:00     4.891     45.770    14
10   73 2015-04-07 15:54:00     4.853     45.738    15
12   72 2015-03-27 18:30:00     4.868     45.751    13
13   69 2015-04-07 10:28:00     4.591     45.820    15
14   75 2015-05-05 10:45:00     4.875     45.780    19
17   75 2015-04-14 08:10:00     4.874     45.784    16
18   53 2015-03-15 04:04:00     4.874     45.785    11
19   65 2015-04-04 05:12:00     4.891     45.770    14
22   65 2015-04-08 10:21:00     4.873     45.783    15
25   59 2015-04-03 15:30:00     5.952     45.558    14
26   59 2015-04-08 06:42:00     4.867     45.751    15
31   67 2015-03-22 17:40:00     4.870     45.776    12
33   67 2015-04-14 02:23:00     4.870     45.776    16
34   73 2015-04-19 07:12:00     4.853     45.739    16
37   72 20

In [3]:
print(df.head(50))

    id                date  latitude  longitude  week
0    1 2015-03-04 00:35:00  4.870147  45.772140    10
1    1 2015-03-04 00:35:00  4.870218  45.772095    10
2    1 2015-03-04 00:35:00  4.870210  45.772072    10
3    1 2015-03-04 00:35:00  4.870210  45.772072    10
4    1 2015-03-04 00:35:00  4.870210  45.772072    10
5    1 2015-03-04 00:35:00  4.870210  45.772072    10
6    1 2015-03-04 00:35:00  4.870210  45.772072    10
7    1 2015-03-04 00:35:00  4.870210  45.772072    10
8    1 2015-03-04 00:35:00  4.870210  45.772072    10
9    1 2015-03-04 00:35:00  4.870210  45.772072    10
10   1 2015-03-04 00:36:00  4.870210  45.772072    10
11   1 2015-03-04 00:36:00  4.870210  45.772072    10
12   1 2015-03-04 00:36:00  4.870210  45.772072    10
13   1 2015-03-04 00:36:00  4.870210  45.772072    10
14   1 2015-03-04 00:36:00  4.870210  45.772072    10
15   1 2015-03-04 00:36:00  4.870210  45.772072    10
16   1 2015-03-04 00:36:00  4.870210  45.772072    10
17   1 2015-03-04 00:36:00  

In [4]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['1' '107' '11' '110' '13' '14' '15' '16' '17' '18' '2' '21' '23' '24'
 '25' '26' '27' '28' '29' '30' '31' '32' '34' '35' '36' '37' '38' '39' '4'
 '41' '42' '43' '44' '48' '49' '5' '50' '51' '52' '53' '54' '55' '58' '59'
 '6' '60' '62' '63' '65' '66' '67' '68' '69' '7' '70' '71' '72' '73' '75'
 '77' '78' '8' '81' '83' '84' '87' '89' '9' '98'] 69


In [5]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [6]:
print(mean_lat_ori)
print(mean_long_ori)
print(mean_date_ori) 
print(count_ligne_ori)
print(mean_hour_ori)

id   week
1    10      4.827873
     11      4.876658
     12      3.315934
     13      4.411797
     14      4.875356
               ...   
107  10      4.870500
     11      4.764978
     12      4.879813
110  10     -0.119641
     11      2.344631
Name: latitude, Length: 449, dtype: float64
id   week
1    10      45.592419
     11      45.781152
     12      44.737925
     13      45.429105
     14      45.774198
               ...    
107  10      45.774327
     11      45.862149
     12      45.786341
110  10      48.404082
     11      48.884047
Name: longitude, Length: 449, dtype: float64
id   week
1    10     2015-03-05 21:39:07.527484928
     11     2015-03-13 14:25:11.006843136
     12     2015-03-19 23:50:54.719796480
     13     2015-03-27 05:09:54.252334592
     14     2015-04-02 16:30:12.192103680
                         ...             
107  10     2015-03-07 04:27:02.706065408
     11     2015-03-13 22:06:22.832617984
     12     2015-03-17 11:12:29.716608000
110  10 

In [7]:
print(mean_lat_anon)
print(mean_long_anon)
print(mean_date_anon) 
print(count_ligne_anon)
print(mean_hour_anon)

id  week
1   10      4.855856
    11      4.877200
    12      3.315147
    13      4.371445
    14      4.876924
              ...   
9   15      4.841287
    16      4.841293
    19      4.841383
98  10      4.869770
    11      4.783538
Name: latitude, Length: 449, dtype: float64
id  week
1   10      45.535227
    11      45.780827
    12      44.728008
    13      45.375112
    14      45.773383
              ...    
9   15      45.771277
    16      45.771319
    19      45.771323
98  10      45.776055
    11      45.876040
Name: longitude, Length: 449, dtype: float64
id  week
1   10     2015-03-05 22:01:10.303199744
    11     2015-03-13 14:06:03.169841408
    12     2015-03-20 00:21:15.784700160
    13     2015-03-27 02:26:12.453398784
    14     2015-04-02 16:39:45.618273792
                        ...             
9   15     2015-04-09 09:30:10.555302144
    16     2015-04-17 06:32:27.072548864
    19     2015-05-06 08:02:54.459499776
98  10     2015-03-07 03:11:22.734374912
 

In [9]:
import numpy as np

for week, group in df.groupby('week'):
    print(f'Week {week}:')
    unique_ids = np.sort(group['id'].unique()) 
    print("Number of unique IDs real:", len(unique_ids))
    print(unique_ids)
    print()

Week 10:
Number of unique IDs real: 64
[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  54  55  58  59  60  62  63  66  67  68  69  71  72
  73  75  77  78  83  87  89  98 107 110]

Week 11:
Number of unique IDs real: 59
[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  21  23  24  25
  26  27  28  29  30  31  34  36  37  38  41  42  43  44  48  49  50  51
  52  53  54  55  58  59  60  62  63  67  68  69  72  73  75  77  78  83
  87  89  98 107 110]

Week 12:
Number of unique IDs real: 51
[  1   2   4   6   7   8   9  13  15  16  17  18  24  26  27  28  30  31
  32  34  38  41  42  43  44  49  50  51  52  53  55  59  60  62  63  65
  66  67  68  69  70  71  72  73  75  77  78  81  84  87 107]

Week 13:
Number of unique IDs real: 48
[ 1  2  4  6  7  8  9 13 17 18 24 26 27 28 29 30 31 32 35 41 42 43 44 49
 50 51 52 53 54 59 60 62 63 65 66 67 68 69 70 71 72 73 75 

In [10]:
dict_id_ori = {
    str("2015-10"): [
            1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 21, 23, 24,
            25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44,
            48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72,
            73, 75, 77, 78, 83, 87, 89, 98, 107, 110
        ],
    str("2015-11"): [
            1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 21, 23, 24, 25,
            26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 41, 42, 43, 44, 48, 49, 50, 51,
            52, 53, 54, 55, 58, 59, 60, 62, 63, 67, 68, 69, 72, 73, 75, 77, 78, 83,
            87, 89, 98, 107, 110
        ],
    str("2015-12"): [
            1, 2, 4, 6, 7, 8, 9, 13, 15, 16, 17, 18, 24, 26, 27, 28, 30, 31,
            32, 34, 38, 41, 42, 43, 44, 49, 50, 51, 52, 53, 55, 59, 60, 62, 63, 65,
            66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 84, 87, 107
        ],
    str("2015-13"): [
            1, 2, 4, 6, 7, 8, 9, 13, 17, 18, 24, 26, 27, 28, 29, 30, 31, 32, 35, 41,
            42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71,
            72, 73, 75, 77, 78, 81, 83, 84
        ],
    str("2015-14"): [
            1, 2, 4, 6, 7, 8, 9, 13, 17, 24, 27, 28, 29, 30, 31, 32, 35, 41, 42, 49,
            50, 51, 52, 53, 54, 59, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 78,
            81, 83, 84
        ],
    str("2015-15"): [
            1, 2, 4, 6, 7, 8, 9, 13, 18, 24, 26, 27, 30, 31, 32, 35, 39, 41, 42, 44,
            49, 50, 51, 59, 62, 63, 65, 66, 67, 68, 69, 71, 72, 73, 75, 81, 83
        ],
    str("2015-16"): [
            1, 2, 4, 6, 7, 8, 9, 13, 18, 24, 26, 27, 30, 31, 32, 35, 36, 39, 41, 42,
            48, 49, 50, 51, 59, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 78, 83, 87
        ],
    str("2015-17"): [
            1, 2, 4, 7, 8, 13, 17, 18, 24, 26, 27, 30, 32, 41, 42, 50, 51, 59, 63, 66,
            67, 68, 69, 71, 72, 73, 75, 77, 78, 83, 87
        ],
    str("2015-18"): [
            1, 2, 4, 17, 18, 24, 26, 30, 32, 36, 39, 42, 50, 51, 59, 62, 63, 66, 67, 68,
            69, 71, 72, 73, 75, 77, 78, 83, 87, 89
        ],
    str("2015-19"): [
            1, 2, 4, 9, 13, 17, 18, 24, 26, 30, 32, 39, 42, 43, 50, 51, 58, 59, 60, 62, 
            63, 66, 67, 68, 69, 72, 73, 75, 83, 87
        ],
    str("2015-20"): [
            1, 2, 4, 13, 18, 24, 42, 50, 51, 59, 62, 66, 67, 68, 69, 72, 73, 75, 77
        ]
}

print(dict_id_ori)

{'2015-10': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-11': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 58, 59, 60, 62, 63, 67, 68, 69, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-12': [1, 2, 4, 6, 7, 8, 9, 13, 15, 16, 17, 18, 24, 26, 27, 28, 30, 31, 32, 34, 38, 41, 42, 43, 44, 49, 50, 51, 52, 53, 55, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 84, 87, 107], '2015-13': [1, 2, 4, 6, 7, 8, 9, 13, 17, 18, 24, 26, 27, 28, 29, 30, 31, 32, 35, 41, 42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 83, 84], '2015-14': [1, 2, 4, 6, 7, 8, 9, 13, 17, 24, 27, 28, 29, 30, 31, 32, 35, 41, 42, 49,

In [13]:
for week, group in anon.groupby('week'):
    print(f'Week {week}:')
    unique_ids_anon = np.sort(group['id'].unique()) 
    print("Number of unique IDs anon:", len(unique_ids_anon))
    print(unique_ids_anon)
    print()

Week 10:
Number of unique IDs anon: 64
['1' '107' '11' '110' '13' '14' '15' '16' '17' '18' '2' '21' '23' '24'
 '25' '26' '27' '28' '29' '30' '31' '32' '34' '35' '36' '37' '38' '39' '4'
 '41' '42' '43' '44' '48' '49' '5' '50' '51' '52' '54' '55' '58' '59' '6'
 '60' '62' '63' '66' '67' '68' '69' '7' '71' '72' '73' '75' '77' '78' '8'
 '83' '87' '89' '9' '98']

Week 11:
Number of unique IDs anon: 59
['1' '107' '11' '110' '13' '14' '15' '16' '17' '2' '21' '23' '24' '25'
 '26' '27' '28' '29' '30' '31' '34' '36' '37' '38' '4' '41' '42' '43' '44'
 '48' '49' '5' '50' '51' '52' '53' '54' '55' '58' '59' '6' '60' '62' '63'
 '67' '68' '69' '7' '72' '73' '75' '77' '78' '8' '83' '87' '89' '9' '98']

Week 12:
Number of unique IDs anon: 51
['1' '107' '13' '15' '16' '17' '18' '2' '24' '26' '27' '28' '30' '31' '32'
 '34' '38' '4' '41' '42' '43' '44' '49' '50' '51' '52' '53' '55' '59' '6'
 '60' '62' '63' '65' '66' '67' '68' '69' '7' '70' '71' '72' '73' '75' '77'
 '78' '8' '81' '84' '87' '9']

Week 13:
Num

In [12]:
dict_id_anon = {
    str("2015-10"): [
            '1', '107', '11', '110', '13', '14', '15', '16', '17', '18', '2', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '4', '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '54', '55', '58', '59', '6', '60', '62', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'
        ],
    str("2015-11"): [
            '1', '107', '11', '110', '13', '14', '15', '16', '17', '2', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '34', '36', '37', '38', '4', '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '53', '54', '55', '58', '59', '6', '60', '62', '63', '67', '68', '69', '7', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'
        ],
    str("2015-12"): [
            '1', '107', '13', '15', '16', '17', '18', '2', '24', '26', '27', '28', '30', '31', '32', '34', '38', '4', '41', '42', '43', '44', '49', '50', '51', '52', '53', '55', '59', '6', '60', '62', '63', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '75', '77', '78', '8', '81', '84', '87', '9'
        ],
    str("2015-13"): [
            '1', '13', '17', '18', '2', '24', '26', '27', '28', '29', '30', '31', '32', '35', '4', '41', '42', '43', '44', '49', '50', '51', '52', '53', '54', '59', '6', '60', '62', '63', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '75', '77', '78', '8', '81', '83', '84', '9'
        ],
    str("2015-14"): [
            '1', '13', '17', '2', '24', '27', '28', '29', '30', '31', '32', '35', '4', '41', '42', '49', '50', '51', '52', '53', '54', '59', '6', '62', '63', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '75', '78', '8', '81', '83', '84', '9'
        ],
    str("2015-15"): [
            '1', '13', '18', '2', '24', '26', '27', '30', '31', '32', '35', '39', '4', '41', '42', '44', '49', '50', '51', '59', '6', '62', '63', '65', '66', '67', '68', '69', '7', '71', '72', '73', '75', '8', '81', '83', '9'
        ],
    str("2015-16"): [
            '1', '13', '18', '2', '24', '26', '27', '30', '31', '32', '35', '36', '39', '4', '41', '42', '48', '49', '50', '51', '59', '6', '62', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '78', '8', '83', '87', '9'
        ],
    str("2015-17"): [
            '1', '13', '17', '18', '2', '24', '26', '27', '30', '32', '4', '41', '42', '50', '51', '59', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '77', '78', '8', '83', '87'
        ],
    str("2015-18"): [
            '1', '17', '18', '2', '24', '26', '30', '32', '36', '39', '4', '42', '50', '51', '59', '62', '63', '66', '67', '68', '69', '71', '72', '73', '75', '77', '78', '83', '87', '89'
        ],
    str("2015-19"): [
            '1', '13', '17', '18', '2', '24', '26', '30', '32', '39', '4', '42', '43', '50', '51', '58', '59', '60', '62', '63', '66', '67', '68', '69', '72', '73', '75', '83', '87', '9'
        ],
    str("2015-20"): [
            '1', '13', '18', '2', '24', '4', '42', '50', '51', '59', '62', '66', '67', '68', '69', '72', '73', '75', '77'
        ]
}

print(dict_id_anon)

{'2015-10': ['1', '107', '11', '110', '13', '14', '15', '16', '17', '18', '2', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '34', '35', '36', '37', '38', '39', '4', '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '54', '55', '58', '59', '6', '60', '62', '63', '66', '67', '68', '69', '7', '71', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'], '2015-11': ['1', '107', '11', '110', '13', '14', '15', '16', '17', '2', '21', '23', '24', '25', '26', '27', '28', '29', '30', '31', '34', '36', '37', '38', '4', '41', '42', '43', '44', '48', '49', '5', '50', '51', '52', '53', '54', '55', '58', '59', '6', '60', '62', '63', '67', '68', '69', '7', '72', '73', '75', '77', '78', '8', '83', '87', '89', '9', '98'], '2015-12': ['1', '107', '13', '15', '16', '17', '18', '2', '24', '26', '27', '28', '30', '31', '32', '34', '38', '4', '41', '42', '43', '44', '49', '50', '51', '52', '53', '55', '59', '6', '60', '62', '63', '65', '66', '67', '68', '69', '7', '70',

In [14]:
for key, value in dict_id_anon.items():
    dict_id_anon[key] = [int(x) for x in ' '.join(map(str, np.sort([int(x) for x in value]))).split()]

In [15]:
print(dict_id_anon)

{'2015-10': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 18, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 48, 49, 50, 51, 52, 54, 55, 58, 59, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-11': [1, 2, 4, 5, 6, 7, 8, 9, 11, 13, 14, 15, 16, 17, 21, 23, 24, 25, 26, 27, 28, 29, 30, 31, 34, 36, 37, 38, 41, 42, 43, 44, 48, 49, 50, 51, 52, 53, 54, 55, 58, 59, 60, 62, 63, 67, 68, 69, 72, 73, 75, 77, 78, 83, 87, 89, 98, 107, 110], '2015-12': [1, 2, 4, 6, 7, 8, 9, 13, 15, 16, 17, 18, 24, 26, 27, 28, 30, 31, 32, 34, 38, 41, 42, 43, 44, 49, 50, 51, 52, 53, 55, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 84, 87, 107], '2015-13': [1, 2, 4, 6, 7, 8, 9, 13, 17, 18, 24, 26, 27, 28, 29, 30, 31, 32, 35, 41, 42, 43, 44, 49, 50, 51, 52, 53, 54, 59, 60, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 78, 81, 83, 84], '2015-14': [1, 2, 4, 6, 7, 8, 9, 13, 17, 24, 27, 28, 29, 30, 31, 32, 35, 41, 42, 49,

In [16]:
weeks = []

for key in dict_id_ori:
    weeks.append(key)

result = {}

for week in weeks:
    pairs = []

    for e in dict_id_ori[week]:
        
        if e in dict_id_anon[week]:
            pairs.append((e, e))
        else:
            pairs.append((e, 'DEL'))

    result[week] = pairs
    print(week, pairs)
    print(len(pairs))

2015-10 [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (52, 52), (54, 54), (55, 55), (58, 58), (59, 59), (60, 60), (62, 62), (63, 63), (66, 66), (67, 67), (68, 68), (69, 69), (71, 71), (72, 72), (73, 73), (75, 75), (77, 77), (78, 78), (83, 83), (87, 87), (89, 89), (98, 98), (107, 107), (110, 110)]
64
2015-11 [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (34, 34), (36, 36), (37, 37), (38, 38), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (52, 52

In [17]:
print(result)

{'2015-10': [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (18, 18), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (32, 32), (34, 34), (35, 35), (36, 36), (37, 37), (38, 38), (39, 39), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (52, 52), (54, 54), (55, 55), (58, 58), (59, 59), (60, 60), (62, 62), (63, 63), (66, 66), (67, 67), (68, 68), (69, 69), (71, 71), (72, 72), (73, 73), (75, 75), (77, 77), (78, 78), (83, 83), (87, 87), (89, 89), (98, 98), (107, 107), (110, 110)], '2015-11': [(1, 1), (2, 2), (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9), (11, 11), (13, 13), (14, 14), (15, 15), (16, 16), (17, 17), (21, 21), (23, 23), (24, 24), (25, 25), (26, 26), (27, 27), (28, 28), (29, 29), (30, 30), (31, 31), (34, 34), (36, 36), (37, 37), (38, 38), (41, 41), (42, 42), (43, 43), (44, 44), (48, 48), (49, 49), (50, 50), (51, 51), (5

In [37]:
attack_TresAnonyme_331_json = {}

for key, value in result.items():
    inner_dict = {}
    for inner_key, inner_value in value:
        inner_dict[inner_key] = [inner_value]
    
    for inner_key, inner_value in value:
        if inner_key in attack_TresAnonyme_331_json:
            attack_TresAnonyme_331_json[inner_key][key] = [str(inner_value)]
        else:
            attack_TresAnonyme_331_json[inner_key] = {key: [str(inner_value)]}

print(attack_TresAnonyme_331_json)

{1: {'2015-10': ['1'], '2015-11': ['1'], '2015-12': ['1'], '2015-13': ['1'], '2015-14': ['1'], '2015-15': ['1'], '2015-16': ['1'], '2015-17': ['1'], '2015-18': ['1'], '2015-19': ['1'], '2015-20': ['1']}, 2: {'2015-10': ['2'], '2015-11': ['2'], '2015-12': ['2'], '2015-13': ['2'], '2015-14': ['2'], '2015-15': ['2'], '2015-16': ['2'], '2015-17': ['2'], '2015-18': ['2'], '2015-19': ['2'], '2015-20': ['2']}, 4: {'2015-10': ['4'], '2015-11': ['4'], '2015-12': ['4'], '2015-13': ['4'], '2015-14': ['4'], '2015-15': ['4'], '2015-16': ['4'], '2015-17': ['4'], '2015-18': ['4'], '2015-19': ['4'], '2015-20': ['4']}, 5: {'2015-10': ['5'], '2015-11': ['5']}, 6: {'2015-10': ['6'], '2015-11': ['6'], '2015-12': ['6'], '2015-13': ['6'], '2015-14': ['6'], '2015-15': ['6'], '2015-16': ['6']}, 7: {'2015-10': ['7'], '2015-11': ['7'], '2015-12': ['7'], '2015-13': ['7'], '2015-14': ['7'], '2015-15': ['7'], '2015-16': ['7'], '2015-17': ['7']}, 8: {'2015-10': ['8'], '2015-11': ['8'], '2015-12': ['8'], '2015-13': 

In [38]:
jsonData = json.dumps(attack_TresAnonyme_331_json, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "1"
        ],
        "2015-11": [
            "1"
        ],
        "2015-12": [
            "1"
        ],
        "2015-13": [
            "1"
        ],
        "2015-14": [
            "1"
        ],
        "2015-15": [
            "1"
        ],
        "2015-16": [
            "1"
        ],
        "2015-17": [
            "1"
        ],
        "2015-18": [
            "1"
        ],
        "2015-19": [
            "1"
        ],
        "2015-20": [
            "1"
        ]
    },
    "2": {
        "2015-10": [
            "2"
        ],
        "2015-11": [
            "2"
        ],
        "2015-12": [
            "2"
        ],
        "2015-13": [
            "2"
        ],
        "2015-14": [
            "2"
        ],
        "2015-15": [
            "2"
        ],
        "2015-16": [
            "2"
        ],
        "2015-17": [
            "2"
        ],
        "2015-18": [
            "2"
        ],
        "2